In [1]:
import os
import sys
import numpy as np
# import numpy.ma as ma
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import pandas as pd
# import cmocean
import xarray as xr
import seaborn as sns

import rasterio
def print_raster(raster):
    print(
        f"shape: {raster.shape}\n"
#         f"resolution: {raster.resolution()}\n"
        f"bounds: {raster.bounds}\n"
#         f"sum: {raster.sum().item()}\n"
        f"CRS: {raster.crs}\n"
    )

/Users/dporter/opt/anaconda3/envs/bedclass/lib/python3.9/site-packages/xarray/backends/cfgrib_.py:27: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(


In [2]:
# Determine where to look #
basedir = '/Users/dporter/data/'
# basedir = 'data'
# basedir = '/Users/dporter/Documents/Research/Projects/bedclass/data'

datadir = 'Antarctic/ROSETTA/for_COMBINED/'

In [3]:
suffix = '.csv'
datafile = 'rosetta_combined'

infile = os.path.join(basedir, datadir, datafile + suffix)
print(infile)

/Users/dporter/data/Antarctic/ROSETTA/for_COMBINED/rosetta_combined.csv


In [4]:
df = pd.read_csv(infile)
df

,Line,X_EPSG_3031,Y_EPSG_3031,unixtime,thickness_dice,rosetta_lidar,rosetta_lidar_wrtgeo,rosetta_lidar_thickness,diff_diceltthick_rsaltthick,dice_thick_bedmapthick
0,L20,-178255.739629,-460417.119902,1.480723e+09,NaN,166.500472,203.484556,1851.169232,NaN,NaN
1,L20,-178346.076341,-460417.397744,1.480723e+09,NaN,166.328215,203.316570,1849.641012,NaN,NaN
2,L20,-178436.480663,-460417.972846,1.480723e+09,NaN,166.155938,203.148576,1848.112705,NaN,NaN
3,L20,-178526.896118,-460418.579014,1.480723e+09,NaN,165.983822,202.980743,1846.585878,NaN,NaN
4,L20,-178617.232081,-460419.255438,1.480723e+09,NaN,165.812069,202.813272,1845.062336,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
646972,T1140,331761.487504,-962391.639624,1.448396e+09,NaN,114.846254,162.627720,1479.480494,NaN,NaN
646973,T1140,331761.575345,-962295.965842,1.448396e+09,NaN,121.094440,168.872927,1536.295304,NaN,NaN
646974,T1140,331761.616047,-962200.289242,1.448396e+09,NaN,127.342182,175.117692,1593.106081,NaN,NaN
646975,T1140,331761.623322,-962104.611879,1.448396e+09,NaN,133.589404,181.361937,1649.912134,NaN,NaN


In [ ]:
print(df.info())
print (df['Line'].unique())

In [ ]:
plt.scatter(df.X_EPSG_3031, df.Y_EPSG_3031, c=df.thickness_dice)


In [ ]:
# ln20 = df.where(df.Line == 'L20')

line = df.query('Line == "L780"')
line

In [ ]:
## fix PROJ path
import pyproj

projpath = sys.prefix + '/share/proj'
pyproj.datadir.set_data_dir(projpath)

In [ ]:
# from osgeo import ogr
import geopandas as gpd
# import shapefile as shp

# data/ASE_catchments_matchGL.geojson
# data/ASE_GroundingLine_simple.geojson

outlines = 'data/ASE_catchments+GL_3031.shp'

shapefile = gpd.read_file(outlines)

shapefile.head()

In [ ]:
from matplotlib.colors import ListedColormap

cmap = np.zeros([256, 4])
cmap[:, 3] = np.linspace(0, 1, 256)
cmap = ListedColormap(cmap)

In [ ]:
fig, ax = plt.subplots(dpi=144)  # figsize=(8, 8), 
ax.set_aspect('equal')

shapefile.boundary.plot(ax=ax, edgecolor='black')

plt.scatter(df.X_EPSG_3031, df.Y_EPSG_3031, c=df.thickness_dice,
            vmin=60, vmax=600,
            marker='.',
            cmap="Spectral_r")
plt.colorbar(label="thickness_dice (m)")

# plt.scatter(df_ASE.X, df_ASE.Y, c=y_pred_ASE, 
#             marker='.',
#             cmap=cmap, edgecolors=None)
plt.scatter(line.X_EPSG_3031[::500], line.Y_EPSG_3031[::500], 
            c='black',
            marker='.',
            cmap=cmap, edgecolors=None,
            )

plt.xlim(-0.8e6, 0.5e6)
plt.ylim(-1.5e6, -0.4e6)

# plt.savefig('figs/mapplot_ASE_ypred_antgg.png')


In [ ]:
# ln20['datetime'] = 
line.index = pd.to_datetime(line["unixtime"], unit='s')
line.tail()

In [ ]:

line[['thickness_dice', 'rosetta_lidar_thickness', 'dice_thick_bedmapthick']].plot()
plt.ylim([-200, 700])
# plt.xlim(pd.Timestamp('2015-12-02 11:10:00'), pd.Timestamp('2015-12-02 12:39:00'))


### Sea level from LIDAR

In [ ]:
# ssh_lidar = gb.get_group('T1090').loc[13900:14100]['rosetta_lidar_wrtgeo'].mean()
#     print(f"Sea level is {ssh_lidar}m")

In [ ]:
line[['rosetta_lidar', 'rosetta_lidar_wrtgeo']].plot()
plt.xlim(pd.Timestamp('2015-12-02 10:30:00'), pd.Timestamp('2015-12-02 11:15:00'))

In [ ]:
start_openwater = '2015-12-02 10:45:00'
end_openwater = '2015-12-02 11:00:00'

start_channel = '2015-12-02 11:20:00'
end_channel = '2015-12-02 11:25:00'

In [ ]:
ssh_lidar = line.loc[start_openwater:end_openwater]['rosetta_lidar'].mean()
# print(line.between_time(pd.Timestamp('2015-12-02 10:10:00'), pd.Timestamp('2015-12-02 11:39:00'))['rosetta_lidar'].mean())

print(line.loc[start_openwater:end_openwater]['rosetta_lidar_wrtgeo'].mean())
print(f"Sea level is {ssh_lidar:.3f}m")

In [ ]:
line['rosetta_lidar_wrtssh'] = line['rosetta_lidar'] - ssh_lidar


In [ ]:
(line['rosetta_lidar_wrtssh'] - line['rosetta_lidar_wrtgeo']).plot()
plt.title('Icesurface difference using SSH_lidar and geoid')
plt.ylabel('Icesurface Diff [m]')

In [ ]:
line.columns

In [ ]:
line['icebase_dice'] = line['rosetta_lidar'] - line['thickness_dice']


In [ ]:
fig, ax = plt.subplots(figsize=(10, 4), dpi=80)

ax.plot(line['X_EPSG_3031'], line['rosetta_lidar'], label='Surface (lidar)', color='orange')
ax.tick_params(axis='y', labelcolor='orange')
ax.set_ylabel('DICE Thick [Pa s m-1]', color='orange', fontsize=16)
ax.set_xlabel('EPSG  [East]', color='black', fontsize=16)
# add_basin_locs()

ax.plot(line['X_EPSG_3031'], line['icebase_dice'], color='cornflowerblue', 
        label='Icebase (DICE)', ls='-', marker='')
# ax.tick_params(axis='y', labelcolor='cornflowerblue')
# ax.set_ylabel('Icebase DICE', color='cornflowerblue', fontsize=16)


# ax4 = ax.twinx()
# ax4.plot(df['Dist'].loc[df['water'] >= 0.1]/1e3, df['water'].loc[df['water'] >= 0.1], color='green', ls='', marker='*')
# ax4.tick_params(axis='y', labelcolor='green')
# ax4.set_ylabel('Water', color='green', fontsize=16)
# ax4.spines['right'].set_position(('axes', 1.2))

plt.ylim([-400, 50])
# plt.xlim(pd.Timestamp('2015-12-02 11:10:00'), pd.Timestamp('2015-12-02 12:39:00'))
plt.legend()
plt.tight_layout()
# plt.savefig('figs/lineplot_UTG_sample_ClassBeta.png')

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4), dpi=80)

ax.plot(line['X_EPSG_3031']/1e3, line['rosetta_lidar_thickness'], color='orange')
ax.tick_params(axis='y', labelcolor='orange')
ax.set_ylabel('DICE Thick [Pa s m-1]', color='orange', fontsize=16)
ax.set_xlabel('EPSG  [km]', color='black', fontsize=16)
# add_basin_locs()

ax3 = ax.twinx()
ax3.plot(line['X_EPSG_3031']/1e3, line['rosetta_lidar_wrtgeo'], color='cornflowerblue', ls='', marker='x')
ax3.tick_params(axis='y', labelcolor='cornflowerblue')
ax3.set_ylabel('Icebase DICE', color='cornflowerblue', fontsize=16)

# ax4 = ax.twinx()
# ax4.plot(df['Dist'].loc[df['water'] >= 0.1]/1e3, df['water'].loc[df['water'] >= 0.1], color='green', ls='', marker='*')
# ax4.tick_params(axis='y', labelcolor='green')
# ax4.set_ylabel('Water', color='green', fontsize=16)
# ax4.spines['right'].set_position(('axes', 1.2))


plt.tight_layout()
# plt.savefig('figs/lineplot_UTG_sample_ClassBeta.png')



## Impact of firn density assumptions on basal channel geometry

In [ ]:
line.columns

In [ ]:
# line['bedmap2_surface_wrtell_SHIFT'] = line['bedmap2_surface_wrtell'].shift(0)
# print(line[['bedmap2_surface_wrtell_SHIFT', 'rosetta_lidar_wrtgeo']].corr())

line[['thickness_dice',
       'rosetta_lidar', 'rosetta_lidar_wrtgeo', 'rosetta_lidar_thickness',
       'diff_diceltthick_rsaltthick', 'dice_thick_bedmapthick']].corr()

In [ ]:
## QAQC edit
# df_sub['icebase_dice_edit'] = np.where(df['icebase_dice']>-500, df['icebase_dice'], df['icebase_dice']+300)

# line['icebase_dice_edit'] = line['icebase_dice']
# line['icebase_dice_edit'][line['icebase_dice']<-500] = line['icebase_dice'] + 284

# line[['icebase_dice_edit', 'rosetta_lidar_wrtgeo']].plot()

In [ ]:
rho_water = 1.025
rho_ice = 0.917
rho_firn = 0.7
# H_firn = 0 
H_firn = 50
H_ave = line.thickness_dice.mean()
rho_ave = (rho_ice * (H_ave - H_firn)/H_ave) + (rho_firn * H_firn/H_ave)
rho_ave

In [ ]:
# =h_geoid/(1-(rho_ave/1.035))
line['thickness_hydro'] = line['rosetta_lidar_wrtgeo']/(1-(rho_ave/rho_water))
line['thickness_hydro'].plot()
plt.ylim([0, 600])

TODO: make this a function

### why `wrtgeo` though?

In [ ]:
line['icebase_hydro'] = line['rosetta_lidar_wrtgeo'] - line['thickness_hydro']

## plot
line.plot(x='X_EPSG_3031', y = ['icebase_dice', 'rosetta_lidar_wrtgeo', 'icebase_hydro'])
plt.ylim([-400, 60])
plt.xlim([-3e5, 3e5])
# plt.xlim(pd.Timestamp('2015-12-02 11:10:00'), pd.Timestamp('2015-12-02 12:39:00'))

##
# fig, ax = plt.subplots(figsize=(10, 4), dpi=80)

# ax.plot(line['X_EPSG_3031'], line['rosetta_lidar'], label='Surface (lidar)', color='orange')
# ax.tick_params(axis='y', labelcolor='orange')
# ax.set_ylabel('DICE Thick [Pa s m-1]', color='orange', fontsize=16)
# ax.set_xlabel('EPSG  [East]', color='black', fontsize=16)
# # add_basin_locs()

# ax.plot(line['X_EPSG_3031'], line['icebase_dice'], color='cornflowerblue', 
#         label='Icebase (DICE)', ls='-', marker='')
# # ax.tick_params(axis='y', labelcolor='cornflowerblue')
# # ax.set_ylabel('Icebase DICE', color='cornflowerblue', fontsize=16)

# # ax4 = ax.twinx()
# # ax4.plot(df['Dist'].loc[df['water'] >= 0.1]/1e3, df['water'].loc[df['water'] >= 0.1], color='green', ls='', marker='*')
# # ax4.tick_params(axis='y', labelcolor='green')
# # ax4.set_ylabel('Water', color='green', fontsize=16)
# # ax4.spines['right'].set_position(('axes', 1.2))

# plt.ylim([-400, 50])
# plt.xlim(pd.Timestamp(start_channel), pd.Timestamp(end_channel))
# plt.legend()
# plt.tight_layout()
# # plt.savefig('figs/lineplot_UTG_sample_ClassBeta.png')

In [ ]:
# rema_file = '/Users/dporter/data/Antarctic/DEM/REMA/SETSM_WV01_20171016_102001006659B400_102001006CCF3100_seg1_2m_v1.0/SETSM_WV01_20171016_102001006659B400_102001006CCF3100_seg1_2m_v1.0_dem.tif'
rema_file = '/Users/dporter/data/Antarctic/DEM/REMA/SETSM_WV01_20151130_1020010048BBA700_1020010048C54400_seg1_2m_v1.0/SETSM_WV01_20151130_1020010048BBA700_1020010048C54400_seg1_2m_v1.0_dem.tif'
src = rasterio.open(rema_file)

In [ ]:
print(src.name)
print(src.mode)
print(src.closed)
print('\n')
print_raster(src)

In [ ]:
for i, dtype, nodataval in zip(src.indexes, src.dtypes, src.nodatavals):
    print(i, dtype, nodataval)

### Convert dataframe to GeoDataFrame (geoPandas)

In [ ]:
## 1
# import geopandas as gpd
# crs = {'init': 'epsg:3031'}
# gdf = gpd.GeoDataFrame(line, crs=crs).set_geometry('geometry')

## 1.2
# from shapely import wkt

# line['geometry'] = line['geometry'].apply(wkt.loads)
# gdf = gpd.GeoDataFrame(line, crs='epsg:4326')


## 2
gdf = gpd.GeoDataFrame(
    line, geometry=gpd.points_from_xy(line.X_EPSG_3031, line.Y_EPSG_3031)
)

In [ ]:
gdf.plot()

In [ ]:
from rasterio.plot import show

## 1
# show(src, 
#      interpolation='none', 
#      vmin=-20, vmax=-5,
#      transform=src.transform,
#      )

## 2
fig, ax = plt.subplots()

# transform rasterio plot to real world coords
extent=[src.bounds[0], src.bounds[2], src.bounds[1], src.bounds[3]]
ax = rasterio.plot.show(src, extent=extent, ax=ax, 
                        interpolation='none', 
                        vmin=-20, vmax=-5, 
                        cmap='viridis',
                       )

gdf.loc[start_channel:end_channel].plot(ax=ax)



In [ ]:
from rasterio.plot import show_hist

fig, (axrgb, axhist) = plt.subplots(1, 2, figsize=(14,7))

show(src, 
     interpolation='none', 
     vmin=-20, vmax=-5,
     transform=src.transform,
     ax=axrgb,
     )
show_hist(
    src, bins=50, lw=0.0, stacked=False, alpha=0.3,
    histtype='stepfilled', title="Histogram",
    ax=axhist,
    )
plt.show()

In [ ]:
coord_list = [(x,y) for x,y in zip(gdf['geometry'].x , gdf['geometry'].y)]

In [ ]:
coord_list[:5]

In [ ]:
gdf['rema'] = [x[0] for x in src.sample(coord_list)]
# print(gdf['rema'].head())

In [ ]:
# gdf.replace(-9999, pd.NA, inplace=True)
gdf.loc[gdf['rema'] == -9999,'rema'] = np.nan

gdf.describe()


In [ ]:
# gdf['rema'].plot()
gdf[['rosetta_lidar', 'rema']].plot()
plt.xlim(pd.Timestamp(start_channel), pd.Timestamp(end_channel))
plt.ylim([-12,-2])


### Appendix

`grouby`

In [ ]:
if Line == 'l790':
    # gb.get_group('T1090').min()
    gb.get_group('T1090').rosetta_lidar_wrtgeo.plot()
    plt.xlim([13500, 14300])
    plt.ylim([-5, 5])
    ssh_lidar = gb.get_group('T1090').loc[13900:14100]['rosetta_lidar_wrtgeo'].mean()
    print(f"Sea level is {ssh_lidar}m")

In [ ]:
# df_sub['thickness_hydro_ave'] = df_sub['rosetta_lidar_wrtgeo']/(1-(rho_ave/1.035))
# df_sub['thickness_hydro_ave_ssh'] = (df_sub['rosetta_lidar_wrtgeo']- ssh_lidar)/(1-(rho_ave/1.035))

df_sub['icebase_hydro'] = df_sub['rosetta_lidar_wrtgeo'] - df_sub['rosetta_lidar_wrtgeo']/(1-(rho_ice/rho_water))
df_sub['icebase_hydro_ssh'] = df_sub['rosetta_lidar_wrtgeo'] - (df_sub['rosetta_lidar_wrtgeo']- ssh_lidar)/(1-(rho_ice/rho_water))

df_sub['icebase_hydro_firn'] = df_sub['rosetta_lidar_wrtgeo'] - df_sub['rosetta_lidar_wrtgeo']/(1-(rho_ave/rho_water))
df_sub['icebase_hydro_firn_ssh'] = df_sub['rosetta_lidar_wrtgeo'] - (df_sub['rosetta_lidar_wrtgeo']- ssh_lidar)/(1-(rho_ave/rho_water))

fig = plt.figure(dpi=150)
df_sub.plot(x='X_EPSG_3031', 
            y=['rosetta_lidar_wrtgeo', 'icebase_dice_edit', 'icebase_hydro_firn', 'icebase_hydro'],
           figsize=(4,7))
plt.title(Line + r': 50m of 0.7$g cm^{-3}$ firn')
plt.ylabel('m.a.s.l.')
plt.legend(["Surface - LIDAR", "Icebase - radar", "Hydrostatic (with firn)", "Hydrostatic (ice only)"], 
           loc='upper center', 
           fontsize=10, 
           shadow=True, 
           bbox_to_anchor=(0.5, 0.83), 
           ncol=1,
          );
plt.xticks(rotation=45)
plt.savefig(f'/Users/dporter/Google Drive (dporter@ldeo.columbia.edu)/Proposals_gdrive/NASA_ROSES_A15_2021/Figures/ROSETTA_{Line}_hydrotest_rho.png')


# print(f"Mean ice thickness diff when using real sea level (not just geoid) \
#       {(df_sub['thickness_hydro_ssh']-df_sub['thickness_hydro']).mean()}")